# Settings Demo Notebook

This notebook demonstrates how to use the Pydantic models for AppSettings and SystemSettings.


### Import the Models


In [1]:
from models import AppSettings, SystemSettings
import json


### Create Settings with Default Values

Let's create both AppSettings and SystemSettings instances using their default values.


In [2]:
# Create AppSettings with default values
app_settings = AppSettings()
print(app_settings)

# Create SystemSettings with default values
system_settings = SystemSettings()
print(system_settings)

{
  "AppSettings": {
    "object_detection": {
      "cnn_network": "DAN",
      "min_object_size": 6,
      "cnn_confidence_threshold": 0.5
    },
    "tracker": {
      "count_min_seen": 10,
      "count_max_unseen": 20
    }
  }
}
{
  "SystemSettings": {
    "nmea": {
      "ip_address": "127.0.0.1",
      "port": 8080
    }
  }
}


### Generate JSON Schemas

Let's generate JSON schemas from our Pydantic models to document the structure.


In [3]:
# Create settings instances
app_settings = AppSettings()
system_settings = SystemSettings()

# Generate schemas using the mixin method
print("Generating schemas...")
app_schema = app_settings.generate_schema(base_path="settings")  # Creates AppSettings.schema.json
system_schema = system_settings.generate_schema(base_path="settings")  # Creates SystemSettings.schema.json

print("\n=== SystemSettings JSON Schema ===")
print(json.dumps(system_schema, indent=2))


Generating schemas...
Schema saved: settings/AppSettings.schema.json
Schema saved: settings/SystemSettings.schema.json

=== SystemSettings JSON Schema ===
{
  "$defs": {
    "NMEA": {
      "description": "NMEA configuration settings",
      "properties": {
        "ip_address": {
          "default": "127.0.0.1",
          "title": "Ip Address",
          "type": "string",
          "updatable": false
        },
        "port": {
          "default": 8080,
          "minimum": 0,
          "title": "Port",
          "type": "integer",
          "updatable": false
        }
      },
      "title": "NMEA",
      "type": "object"
    }
  },
  "description": "System settings model",
  "properties": {
    "nmea": {
      "$ref": "#/$defs/NMEA"
    }
  },
  "title": "SystemSettings",
  "type": "object"
}


### Connect Generated Schemas to VS Code

To enable JSON validation and autocomplete in VS Code using the generated schemas, add the following to your `.vscode/settings.json` file:

```json
{
  "json.schemas": [
    {
      "fileMatch": ["AppSettings*.json"],
      "url": "file:/path/to/your/project/AppSettings.schema.json"
    },
    {
      "fileMatch": ["SystemSettings*.json"],
      "url": "file:/path/to/your/project/SystemSettings.schema.json"
    }
  ]
}
```

### Benefits:
- ✅ **Autocomplete** - VS Code will suggest valid field names and values
- ✅ **Validation** - Invalid JSON will be highlighted with errors
- ✅ **Hover info** - Hover over fields to see descriptions and types
- ✅ **IntelliSense** - Smart suggestions based on the schema

### File Matching Patterns:
- `AppSettings*.json` - Matches `AppSettings.json`, `AppSettings.brain.json`, `AppSettings.user.json`, etc.
- `SystemSettings*.json` - Matches `SystemSettings.json`, `SystemSettings.user.json`, etc.

### Example with Relative Paths:
```json
{
  "json.schemas": [
    {
      "fileMatch": ["AppSettings*.json"],
      "url": "./schemas/AppSettings.schema.json"
    },
    {
      "fileMatch": ["SystemSettings*.json"],
      "url": "./schemas/SystemSettings.schema.json"
    }
  ]
}
```

This will make editing your settings JSON files much more pleasant with full IDE support!


### Load Overrides
Lets override the default values with custom user of product overrides

In [4]:
system_settings = SystemSettings()
system_settings = SystemSettings.load_with_overrides("settings/SystemSettings.user.json")
# Lets teh changes in the SystemSettings.user.json file were applied
# "nmea": {
#     "ip_address": "192.168.1.100",
#     "port": 2203
#   }
print(system_settings)

app_settings = AppSettings()
app_settings = AppSettings.load_with_overrides("settings/AppSettings.brain.json")

# Lets say the brain is too slow and we want to change the min object size
# "object_detection": {
#     "cnn_confidence_threshold": 0.1,
#     "min_object_size": 0
#   }

print(app_settings)

{
  "SystemSettings": {
    "nmea": {
      "ip_address": "192.168.1.100",
      "port": 2203
    }
  }
}
{
  "AppSettings": {
    "object_detection": {
      "cnn_network": "DAN",
      "min_object_size": 6,
      "cnn_confidence_threshold": 0.9
    },
    "tracker": {
      "count_min_seen": 10,
      "count_max_unseen": 20
    }
  }
}


### Updatable Fields Demo

Let's demonstrate how the `updatable` field parameter works. Some fields can be overridden by user data, while others are protected.

First, let's see which fields are updatable:


#### Update the CNN confidence threshold (updateable)

In [5]:
# Fake a change to the AppSettings Pydantic model
# Lets say the confidence threshold is too low and we want to change it from 0.5 to 0.9
app_settings = AppSettings()
app_settings.object_detection.cnn_confidence_threshold = 0.9

# Update the AppSettings.brain.json file
app_settings.update("settings/AppSettings.brain.json")

# Lets verify the AppSettings.brain.json file was updated
with open("settings/AppSettings.brain.json", "r") as f:
    print(f.read())


{
  "object_detection": {
    "cnn_confidence_threshold": 0.9,
    "min_object_size": 6
  }
}


#### Try to update the user network settings (non-updateable)

In [6]:
system_settings = SystemSettings()
 
# Fake a change to the SystemSettings Pydantic model
# Lets say the default ip addres should be localhost instead 127.0.0.1 and the port should be 3000 instead of 8080
system_settings.nmea.port = 3000
system_settings.nmea.ip_address = "localhost"

# Save the updated settings
system_settings.update("settings/SystemSettings.user.json")

# Let's verify the SystemSettings.user.json file was updated --> it was not updated because the ip_address and port are not updatable
with open("settings/SystemSettings.user.json", "r") as f:
    print(f.read())

Field is not updatable: annotation=str required=False default='127.0.0.1' json_schema_extra={'updatable': False}
Field is not updatable: annotation=int required=False default=8080 json_schema_extra={'updatable': False} metadata=[Ge(ge=0)]
{
  "nmea": {
    "ip_address": "192.168.1.100",
    "port": 2203
  }
}


### Calculate checksum
Lets generate the checksum of a model to see if something has changed

In [12]:
system_settings = SystemSettings()
print(system_settings.checksum())

# # Fake a change to the SystemSettings Pydantic model
# Lets say the port should be 3000 instead of 8080
system_settings.nmea.port = 3000

print(system_settings.checksum())

932b35f0af958b60f5993c5e10cf941585180af34f111d1278b54de3348045c6
0bb0a5d5469956e80f33b984d60e2d4089c96d2d510e0b4631c720101f216a6f
